# FT用データ生成スクリプト

In [26]:
# !conda install -y -c conda-forge kalpy \
# kaldi \
# pynini

# # パッケージインストール
# !pip install -r requirements.sbv.txt

In [27]:
# !pip list

In [28]:
# # mfa
# # 日本語辞書のダウンロード
# !mfa model download dictionary japanese_mfa

# # 日本語音響モデルのダウンロード
# !mfa model download acoustic japanese_mfa

## テキスト対話データ生成

In [29]:
import os
from typing import Literal
import ast

from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# .envファイル読み込み
load_dotenv()

True

In [30]:
#config
from os.path import join, expanduser

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gemini-2.5-flash'
TEMPERATURE = 1.0
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

# 生成する音声のサンプリングレート
setting_sr = 16000

#対話音声データの個数を指定
gen_dial_num = 100

# すでに作成した対話データを削除するかどうか
IS_REMOVE_EXIST_FILE = True

# ftに使うjsonとaudioの出力フォルダパス
home_dir = expanduser("~")
json_dir_path = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/sbv/transcription")
audio_dir_path = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/sbv/audio")

# mfa関連のパス
model_dir = join(home_dir, "Documents/MFA/pretrained_models/acoustic/japanese_mfa.zip")
mfa_input_dir = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/sbv/mfa_input")
mfa_output_dir = join(home_dir, "Github/jmoshi-ft/gen_dialogue/data/sbv/mfa_output")

In [31]:
base_paths = [
    json_dir_path,
    audio_dir_path,
    mfa_input_dir,
    mfa_output_dir,
]

for p in base_paths:
    if not os.path.isdir(p):
        os.makedirs(p)

In [32]:
# model定義
model = ChatGoogleGenerativeAI(
                 model=MODEL,
                 temperature=TEMPERATURE)

# 埋め込みモデル定義
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-large"
)

# データベース定義
vector_store = Chroma(
    collection_name="collection",
    embedding_function=embeddings,
    # persist_directory = "/path/to/db_file" # if necessary
)

In [33]:
loader = DirectoryLoader(
    "../../mental_docs/",
    glob="*.pdf",
    show_progress=True,
    loader_cls=PDFMinerLoader,
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

  0%|                                                                                     | 0/3 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set g

Loaded 3 documents


In [34]:
# Debug
# for doc in docs:
#     print("-------------------------------------------------")
#     print(doc.metadata)
#     print(len(doc.page_content))
#     print(doc.page_content[:100])

In [35]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True, # 分割前の文章のインデックスを追跡
)
splits = text_splitter.split_documents(docs)

# データベースにデータを追加
document_ids = vector_store.add_documents(documents=splits)

In [36]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=2)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

In [37]:
from typing import Literal

from pydantic import BaseModel, Field


class Dialogue(BaseModel):
    """対話データを構成する対話クラス"""
    speaker: Literal["A", "B"] = Field(..., description="話者。Aはカウンセラー、Bはクライエントを表す。")
    text: str = Field(..., description="話者が話した内容。")

class Dialogues(BaseModel):
    """カウンセリングを目的としたカウンセリング対話データ"""
    dialogues: list[Dialogue] = Field(..., description="対話データを構成する対話クラスのリスト。")

In [38]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model, 
    tools=[],
    middleware=[prompt_with_context],
    response_format=ToolStrategy(
        Dialogues,
        handle_errors="フォーマットに合うように、もう一度対話データを生成してください。"
    )
)

In [39]:
#promptを作成
import random


sessions = [
    "【段階：初期】信頼関係を築きつつ、悩みの背景を深掘りするシーン",
    "【段階：中期】クライエントの「すべき思考」に焦点を当て、認知の歪みを扱うシーン",
    "【段階：終結期】これまでのセッションを振り返り、終結に向けて準備するシーン",
]

def gen_prompt_txt():
    choiced = random.randint(0, 2)
    choiced_session = sessions[choiced]
    prompt_txt = f"""メンタルヘルスケアカウンセリングのセッションをシミュレーションしてください。
シミュレーションしたい「段階」と「テーマ」:
{choiced_session}

役割定義:
A (カウンセラー): メンタルヘルスケアの専門知識を持つ経験豊富なカウンセラー。傾聴と共感の姿勢を基本とし、クライエントの言葉を促すように、優しく、自然な話し言葉（「〜ですね」「〜でしたか」など）を使います。
B (クライエント): 仕事上の悩みだけでなく、日常生活全般に対して漠然とした不安や焦りを感じている人物。

対話の要件:
スタイル: 実際の会話の文字起こしのように、堅苦しくない自然な「話し言葉」を使用してください。
相槌 (あいづち): カウンセラー（A）は、クライエント（B）の話を促し、共感を示すため、「ええ」「はい」「そうなんですね」「なるほど」といった細かな相槌を頻繁に、適切なタイミングで挿入してください。
構成: 会話が途中で途切れるのではなく、初回のヒアリングとして「一区切り」がつき、自然に終了する流れにしてください（例：次回の約束、今回のまとめなど）。
分量: 会話の往復は合計12〜20ターン程度、全体の文字数が合計500〜800文字程度になるように構成してください。
"""
    return prompt_txt

In [40]:
# テキスト対話生成関数
def gen_txt_dialogue():
    prompt = gen_prompt_txt()
    resp = agent.invoke({"messages": [{"role": "user", "content": prompt}]})
    dialogues_list = resp["structured_response"].dialogues
    return dialogues_list

In [41]:
#DEBUG
# txt_dialogue = gen_txt_dialogue()
# print(txt_dialogue)
# lst_dialogue = txt_to_lst(txt_dialogue)
# print(lst_dialogue)

## テキスト対話データを音声対話データに変換 

In [42]:
from style_bert_vits2.nlp import bert_models
from style_bert_vits2.constants import Languages
from pathlib import Path
from huggingface_hub import hf_hub_download
from style_bert_vits2.tts_model import TTSModel

bert_models.load_model(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")
bert_models.load_tokenizer(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")
assets_root = Path("model_assets")

# # 子春音あみ
# model_file = "koharune-ami/koharune-ami.safetensors"
# config_file = "koharune-ami/config.json"
# style_file = "koharune-ami/style_vectors.npy"
# hf_repo = "litagin/sbv2_koharune_ami"

# # あみたろ
# model_file = "amitaro/amitaro.safetensors"
# config_file = "amitaro/config.json"
# style_file = "amitaro/style_vectors.npy"
# hf_repo = "litagin/sbv2_amitaro"


# デフォルトの女性2
model_file = "jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors"
config_file = "jvnv-F2-jp/config.json"
style_file = "jvnv-F2-jp/style_vectors.npy"
hf_repo = "litagin/style_bert_vits2_jvnv"

for file in [model_file, config_file, style_file]:
    print(file)
    hf_hub_download(hf_repo, file, local_dir="model_assets")

A_model = TTSModel(
    model_path=assets_root / model_file,
    config_path=assets_root / config_file,
    style_vec_path=assets_root / style_file,
    device="cuda",
)

# デフォルトの男性2
model_file = "jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors"
config_file = "jvnv-M2-jp/config.json"
style_file = "jvnv-M2-jp/style_vectors.npy"

for file in [model_file, config_file, style_file]:
    print(file)
    hf_hub_download(hf_repo, file, local_dir="model_assets")

B_model = TTSModel(
    model_path=assets_root / model_file,
    config_path=assets_root / config_file,
    style_vec_path=assets_root / style_file,
    device="cuda",
)

jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors
jvnv-F2-jp/config.json
jvnv-F2-jp/style_vectors.npy
jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors
jvnv-M2-jp/config.json
jvnv-M2-jp/style_vectors.npy


In [43]:
from typing import Literal

def sbv_tts(text: str, speaker: Literal["A", "B"], assist_text=None):
    if speaker == "A":
        sr, audio = A_model.infer(
            text = text,
            style='Happy',
            style_weight=1,
            split_interval = 0.3,
            use_assist_text = True if assist_text is not None else None,
            assist_text = assist_text
        )
    else:
        sr, audio = B_model.infer(
            text = text,
            style='Sad',
            style_weight=1,
            split_interval = 0.3,
            use_assist_text = True if assist_text is not None else None,
            assist_text = assist_text
        )
    
    return sr, audio

In [44]:
import librosa
import numpy as np

def gen_audio_dialogue(text_dialogue_list):
    # 音声ファイルを順番に生成（ファイルは不要なのでwave配列で持つ）
    wav_data = []
    for dial in text_dialogue_list:
        speaker = dial.speaker
        sr, wav = sbv_tts(dial.text, speaker)
        print(wav.shape)

        # サンプリングレートを変換
        if sr != setting_sr:
            # 16ビット整数のデータを、-1.0から1.0の範囲に収まる浮動小数点数に正規化
            wav = wav.astype(np.float32) / 32768.0
            wav = librosa.resample(wav, orig_sr=sr, target_sr=setting_sr)

        # 0.3秒間の無音時間を追加
        duration_sec = 0.3
        num_silent_samples = int(setting_sr*duration_sec)
        silence = np.zeros(num_silent_samples, dtype=wav.dtype)
        wav_with_silence = np.concatenate((wav, silence))
        wav_data.append(wav_with_silence)
    
    # 最終的な音声長を決定
    max_len = sum([len(w) for w in wav_data])
    
    # ステレオ音声用（2チャンネル×最大長）の空配列をゼロ初期化で作成
    stereo = np.zeros((2, max_len), dtype=np.float32)
    
    pos = 0
    for i, wav in enumerate(wav_data):
        ch = i%2  # 0:左(A), 1:右(B)
        stereo[ch, pos:pos+len(wav)] += wav
        pos += len(wav)
    
    # 転置(-1,2)する
    stereo = stereo.T
    return stereo

## mfa(montreal force alignment)による音声アラインメント

In [45]:
import MeCab
import re

# 句読点のパターン
PUNCT_RE = re.compile(r'^[。、,.!?！？…]+$')

def tokenize_text(text, is_punct_isolated=False):
    tokens = []
    punct_dict = {}
    checked_punct_pos = 0
    try:
        # MeCabのタガーを初期化
        tagger = MeCab.Tagger()

        # MeCabは内部でShift-JISやEUC-JPを期待することがあるため、
        # UnicodeDecodeErrorを避けるために明示的にUTF-8でエンコード・デコードする
        # parseToNodeは、より詳細な情報をノードオブジェクトとして取得できるメソッド
        node = tagger.parseToNode(text)
        while node:
            if not node.surface:
                pass
                
            elif not is_punct_isolated and PUNCT_RE.match(node.surface) and tokens:
                punct_dict[checked_punct_pos] = node.surface
                
                checked_punct_pos += len(node.surface)
                # 句読点なら直前トークンに連結
                tokens[-1] += node.surface
            else:
                checked_punct_pos += len(node.surface)
                
                # 通常トークンはそのまま追加
                tokens.append(node.surface)
            node = node.next
    except RuntimeError as e:
        print(f"MeCabの実行中にエラーが発生しました: {e}", file=sys.stderr)
        
    return tokens, punct_dict

In [46]:
def generate_txt_file_using_mecab(input_txt, path):
    tokens, punct_dict = tokenize_text(input_txt)
    output = ""
    for token in tokens:
        output += token + "\n"
        
    with open(path, "w", encoding="utf-8") as f:
        f.write(output)
    return tokens, punct_dict

In [47]:
from os.path import join, expanduser
import subprocess
import json

def alignment_channel(channel, txt, target_dir_name):
    input_dir_path = join(mfa_input_dir, target_dir_name)
    output_dir_path = join(mfa_output_dir, target_dir_name)
    os.makedirs(input_dir_path, exist_ok=True)
    os.makedirs(output_dir_path, exist_ok=True)
    
    for_align_audio_path = join(input_dir_path, f"{target_dir_name}.wav")
    for_align_txt_path = join(input_dir_path, f"{target_dir_name}.txt")

    sf.write(for_align_audio_path, channel, setting_sr)
    _, punct_dict = generate_txt_file_using_mecab(txt, for_align_txt_path)
    subprocess.run([
        "mfa",
        "align",
        input_dir_path,
        "japanese_mfa",
        model_dir,
        output_dir_path,
        "--",
        "--overwrite",
        "--clean",
        "----final_clean",
        "--output_format", "json",
        "--beam", "1000",
        "--retry_beam", "4000",
        "--punctuation", "…",
    ])
    return punct_dict

def json_formatter_for_ft(align_json_A, align_json_B):
    json = []

    segments_A = align_json_A["tiers"]["words"]["entries"]
    segments_B = align_json_B["tiers"]["words"]["entries"]
    for segment in segments_A:
        json.append({
            "speaker": "A",
            "word": segment[2],
            "start": segment[0],
            "end": segment[1],
        })
    for segment in segments_B:
        json.append({
            "speaker": "B",
            "word": segment[2],
            "start": segment[0],
            "end": segment[1],
        })
    sorted_json = sorted(json, key=lambda seg: seg["start"])
    return sorted_json

def lst_to_line_str(lst):
    result = ""
    for s in lst:
        result += s
    return result
    
def alignment_audio_dialogue(text_dialogue_list, audio_path, idx):
    # ステレオ分離: speaker A=左(0), B=右(1)と仮定
    audio, sr = sf.read(audio_path)    # (samples, channels)
    channel_A = audio[:,0]
    channel_B = audio[:,1]
    txt_lst_A = []
    txt_lst_B = []
    for txt_dial in text_dialogue_list:
        if txt_dial.speaker == "A":
            txt_lst_A.append(txt_dial.text)
        else:
            txt_lst_B.append(txt_dial.text)
    A_full_txt = lst_to_line_str(txt_lst_A)
    B_full_txt = lst_to_line_str(txt_lst_B)
    
    target_dir_name_A = f"A_{idx}"
    target_dir_name_B = f"B_{idx}"
    punct_dict_A = alignment_channel(channel_A, A_full_txt, target_dir_name_A)
    punct_dict_B = alignment_channel(channel_B, B_full_txt, target_dir_name_B)
    json_path_A = join(mfa_output_dir, target_dir_name_A, f"{target_dir_name_A}.json")
    json_path_B = join(mfa_output_dir, target_dir_name_B, f"{target_dir_name_B}.json")
    with open(json_path_A, "r") as f:
        json_A = json.load(f)
    with open(json_path_B, "r") as f:
        json_B = json.load(f)
    
    ft_json = json_formatter_for_ft(json_A, json_B)

    return ft_json

## フォルダ初期化

In [48]:
import re

def get_file_name():
    wav_file_pattern = r"^(\d+)\.wav$"
    num = -1
    for file in os.listdir(audio_dir_path):
        if not os.path.exists(os.path.join(audio_dir_path, file)):
            continue
        if not re.match(wav_file_pattern, file):
            continue
    
        match_obj = re.match(wav_file_pattern, file)
        get_number = int(match_obj.groups()[0])
    
        if num < get_number:
            num = get_number
    return num

In [49]:
from glob import glob
import shutil

def delete_files(dir_path):
    shutil.rmtree(dir_path)
    os.makedirs(dir_path)

if IS_REMOVE_EXIST_FILE:
    file_name_num = -1
    for dir_path in base_paths:
        delete_files(dir_path)
else:
    file_name_num = get_file_name()

## メイン処理

In [50]:
%%time

import soundfile as sf
import json

for i in range(file_name_num+1, gen_dial_num+file_name_num+1):

    txt_dialogue_list = gen_txt_dialogue()
    stereo = gen_audio_dialogue(txt_dialogue_list)
    
    wav_name = f"{i}.wav"
    audio_file_path = os.path.join(audio_dir_path, wav_name)

    # wavファイル出力
    sf.write(audio_file_path, stereo, setting_sr)

    json_data = alignment_audio_dialogue(txt_dialogue_list, audio_file_path, i)

    json_name = f"{i}.json"
    json_file_path = os.path.join(json_dir_path, json_name)
    
    # JSON出力
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

10-25 19:44:40 |  INFO  | tts_model.py:259 | Start generating audio data from text:
今日は、お越しくださってありがとうございます。何か、今お話ししたいこと、ありますか？
10-25 19:44:40 |  INFO  | infer.py:24 | Using JP-Extra model


/home1/s1f102201582/anaconda3/envs/sbv-tts/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


10-25 19:44:41 |  INFO  | safetensors.py:50 | Loaded 'model_assets/jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors' (iteration 166)
10-25 19:44:41 |  INFO  | tts_model.py:324 | Audio data generated successfully
(275968,)
10-25 19:44:41 |  INFO  | tts_model.py:259 | Start generating audio data from text:
はい、ありがとうございます。最近、仕事のことや、日常生活全体に漠然とした不安や焦りを感じることが多くて…。
10-25 19:44:41 |  INFO  | infer.py:24 | Using JP-Extra model
10-25 19:44:42 |  INFO  | safetensors.py:50 | Loaded 'model_assets/jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors' (iteration 159)
10-25 19:44:42 |  INFO  | tts_model.py:324 | Audio data generated successfully
(401408,)
10-25 19:44:42 |  INFO  | tts_model.py:259 | Start generating audio data from text:
そうなんですね。漠然とした不安や焦り、ですか。ええ。どんな時に強く感じられますか？
10-25 19:44:42 |  INFO  | tts_model.py:324 | Audio data generated successfully
(324096,)
10-25 19:44:42 |  INFO  | tts_model.py:259 | Start generating audio data from text:
うーん、仕事中はもちろんですが、家に帰っても落ち着かなくて。夜も寝付けず、朝起きるのもつらい、みたいな感じです。
10-25 19:4

 INFO     Setting up corpus information...                                      
 INFO     Loading corpus from source files...                                   


   1% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/100  [ 0:00:01 < -:--:-- , ? it/s ]


...

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1  [ 0:00:05 < 0:00:00 , ? it/s ]


 INFO     Analyzing alignment quality...                                        


 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1  [ 0:00:01 < 0:00:00 , ? it/s ]
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1  [ 0:00:00 < 0:00:00 , ? it/s ]


 INFO     Exporting alignment TextGrids to                                      
          /users/s1f102201582/Github/jmoshi-ft/gen_dialogue/data/sbv/mfa_output/
          B_99...                                                               
 INFO     Finished exporting TextGrids to                                       
          /users/s1f102201582/Github/jmoshi-ft/gen_dialogue/data/sbv/mfa_output/
          B_99!                                                                 
 INFO     Done! Everything took 155.041 seconds                                 


CPU times: user 44min 21s, sys: 1min 16s, total: 45min 37s
Wall time: 9h 10min 33s
